# Google colab initialization

For Google colab uncomment these lines and run them to access your drive or try the second way (not tested)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
#
# sys.path.insert(1, r'/content/drive/My Drive/')

In [2]:
# Other try
# !git clone https://github.com/Alexanderstaehle/OM_project

In [3]:
# import sys
# sys.path.append("OM_project")

In [4]:
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from utils import ml_utils, visualization, data_loading, tf_models

2022-02-19 13:36:31.890918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 13:36:31.890934: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
data_loading.initialize_env()
sns.set_theme()
color_map = sns.color_palette(as_cmap=True)
ml_utils.check_tpu_gpu()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of accelerators:  1


2022-02-19 13:36:33.145485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 13:36:33.146317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 13:36:33.146453: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-19 13:36:33.146572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-19 13:36:33.146690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

# Different batch sizes with fixed learning rate

In [6]:
models_dict_fixed = {}
batch_sizes = [32, 64, 128, 256, 512, 1024]
# batch_sizes = [32, 64, 128]
lr = 0.001
training_epochs = 10

In [9]:
# test of SAM model 

batch_size = 32

train, validation = data_loading.load_batched_and_resized_dataset(dataset_name='MNIST', batch_size=batch_size,
                                                                 img_size=32)

optimizer = keras.optimizers.SGD(learning_rate=lr)
#base_model = tf_models.build_simple_dense_model(train)
base_model = tf_models.build_simple_cnn(train)
base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
train_callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10,
            restore_best_weights=True
        ),
        # tf.keras.callbacks.ReduceLROnPlateau(
        #     monitor="val_loss", factor=0.5,
        #     patience=3, verbose=1
        # )
    ]
model = tf_models.SAMModel(base_model, adaptive = True, rho = 2.0)    
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#models_dict_fixed[batch_size] = ml_utils.train_model(model, train, validation, epochs=1,
#                                                         extra_callbacks=train_callbacks, verbose=1)
models_dict_fixed[batch_size] = ml_utils.train_model(base_model, train, validation, epochs=1,
                                                         extra_callbacks=train_callbacks, verbose=1)

1500/1500 [==============================] - 20s 13ms/step - loss: 1.9340 - accuracy: 0.3889 - val_loss: 1.2981 - val_accuracy: 0.7424


In [ ]:
base_model.set_weights(model.get_weights())
base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#base_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
base_model.summary()

In [ ]:
for x, y in train:
    print(tf.argmax(base_model(x), axis = -1))
    print(tf.convert_to_tensor(y, dtype=tf.float64))
    print(tf.keras.losses.binary_crossentropy(y, tf.argmax(base_model(x), axis = -1)))
    break

In [10]:
visualization.get_sharpness(base_model, train)

1500/1500 [==============================] - 5s 3ms/step - loss: 1.2936 - accuracy: 0.7446
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        59786     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.29358D+00    |proj g|=  1.20978D-02
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3028 - accuracy: 0.1021



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.


44.00377090533849


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
59786      1      2  53961     0 53960   1.210D-02  -1.294D+00
  F =  -1.2935798168182373     

ABNORMAL_TERMINATION_IN_LNSRCH                              


 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [ ]:
for batch_size in batch_sizes:
    # Read training data
    train, validation = data_loading.load_batched_and_resized_dataset(dataset_name='MNIST', batch_size=batch_size,
                                                                      img_size=32)
    model = tf_models.build_simple_dense_model(train)
    train_callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10,
            restore_best_weights=True
        ),
        # tf.keras.callbacks.ReduceLROnPlateau(
        #     monitor="val_loss", factor=0.5,
        #     patience=3, verbose=1
        # )
    ]
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    # Need sparse categorical crossentropy since our labels are in form of integers not vectors
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    models_dict_fixed[batch_size] = ml_utils.train_model(model, train, validation, epochs=10,
                                                         extra_callbacks=train_callbacks, verbose=1)

In [ ]:
ml_utils.save_model_state(models_dict_fixed, 'model_fixed_lr_diff_bs')

In [ ]:
visualization.plot_loss_by_param(models_dict_fixed, 'batch size with fixed learning rate', 'fixed_lr_diff_bs')


# Different batch sizes with linear increasing learning rate

In [ ]:
learning_rates = [0.001, 0.002, 0.004, 0.008, 0.016, 0.032]